# Data Preparation and EDA for IoT23 dataset

In [1]:
import os 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import csv
from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import time as timer

In [4]:
data= pd.read_csv('iot23_combined_new_3M.csv', low_memory=False)

In [5]:
data.head()  

,Unnamed: 0,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,detailed_label
0,0,1.536227e+09,CeqqKl3hyLQmO8LK98,192.168.100.111,17576.0,78.1.220.212,8081.0,tcp,-,3e-06,...,-,-,0.0,S,2.0,80.0,0.0,0.0,POHScan,NaN
1,1,1.536227e+09,C2oHQWo1EFGH8D9x7,192.168.100.111,17576.0,152.84.7.111,8081.0,tcp,-,2e-06,...,-,-,0.0,S,2.0,80.0,0.0,0.0,POHScan,NaN
2,2,1.536227e+09,CJLVjs4BByG04mczXc,192.168.100.111,17576.0,173.36.41.67,8081.0,tcp,-,2e-06,...,-,-,0.0,S,2.0,80.0,0.0,0.0,POHScan,NaN
3,3,1.536227e+09,C0z4uS9AWHDH2s4S7,192.168.100.111,17576.0,87.13.21.104,8081.0,tcp,-,2e-06,...,-,-,0.0,S,2.0,80.0,0.0,0.0,POHScan,NaN
4,4,1.536227e+09,CxbNVk3liFNUIlqSPi,192.168.100.111,17576.0,99.110.163.140,8081.0,tcp,-,2e-06,...,-,-,0.0,S,2.0,80.0,0.0,0.0,POHScan,NaN


In [6]:
del data['Unnamed: 0']

In [8]:
data.tail()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,detailed_label
2803565,1.562165e+09,CbiAr234EfpzQInxbg,0.0.0.0,68.0,255.255.255.255,67.0,udp,dhcp,90.034713,3300.0,...,-,-,0.0,D,11.0,3608.0,0.0,0.0,Benign,NaN
2803566,1.562165e+09,CqEhZf1lYmVokqECn3,::,143.0,ff02::16,0.0,icmp,-,90.39997,340.0,...,-,-,0.0,-,9.0,844.0,0.0,0.0,Benign,NaN
2803567,1.562165e+09,Cr5vKO1CGfMQwFp22c,::,135.0,ff02::1:ffd5:cdf,136.0,icmp,-,89.82403000000001,72.0,...,-,-,0.0,-,3.0,216.0,0.0,0.0,Benign,NaN
2803568,1.562165e+09,CQ5cJ21U9NUbUWIznd,fe80::5bcc:698e:39d5:cdf,143.0,ff02::16,0.0,icmp,-,45.215915,200.0,...,-,-,0.0,-,8.0,648.0,0.0,0.0,Benign,NaN
2803569,1.562165e+09,CjD5ve2qwxfFtvd4zl,fe80::5bcc:698e:39d5:cdf,133.0,ff02::2,134.0,icmp,-,44.242222999999996,0.0,...,-,-,0.0,-,2.0,96.0,0.0,0.0,Benign,NaN


# Cleaning Process start! 

# dropping unncessary attributes ! 

In [9]:
df= data.drop(columns=['uid','id.orig_h','id.resp_h','local_orig','local_resp','detailed_label'])

In [10]:
df.head()

,ts,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1.536227e+09,17576.0,8081.0,tcp,-,3e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHScan
1,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHScan
2,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHScan
3,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHScan
4,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHScan


In [15]:
df['orig_bytes'] = df['orig_bytes'].str.replace('-','0')
df['resp_bytes'] = df['resp_bytes'].str.replace('-','0')
df['duration'] = df['duration'].str.replace('-','0')

In [16]:
df.isnull().sum()

ts               0
id.orig_p        0
id.resp_p        0
proto            0
service          0
duration         0
orig_bytes       0
resp_bytes       0
conn_state       0
missed_bytes     0
history          0
orig_pkts        0
orig_ip_bytes    0
resp_pkts        0
resp_ip_bytes    0
label            0
dtype: int64

Let us separate the target and independent variables 

okkkk... so there are 7 target variables with very few samples.. we would like to remove all those 

In [17]:
df=df.dropna()

In [18]:
X=df

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2803570 entries, 0 to 2803569
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ts             float64
 1   id.orig_p      float64
 2   id.resp_p      float64
 3   proto          object 
 4   service        object 
 5   duration       object 
 6   orig_bytes     object 
 7   resp_bytes     object 
 8   conn_state     object 
 9   missed_bytes   float64
 10  history        object 
 11  orig_pkts      float64
 12  orig_ip_bytes  float64
 13  resp_pkts      float64
 14  resp_ip_bytes  float64
 15  label          object 
dtypes: float64(8), object(8)
memory usage: 363.6+ MB


In [20]:
X['orig_bytes']=X['orig_bytes'].astype(float)

In [21]:
X['duration']=X['duration'].astype(float)

In [22]:
X['resp_bytes']=X['resp_bytes'].astype(float)

In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2803570 entries, 0 to 2803569
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ts             float64
 1   id.orig_p      float64
 2   id.resp_p      float64
 3   proto          object 
 4   service        object 
 5   duration       float64
 6   orig_bytes     float64
 7   resp_bytes     float64
 8   conn_state     object 
 9   missed_bytes   float64
 10  history        object 
 11  orig_pkts      float64
 12  orig_ip_bytes  float64
 13  resp_pkts      float64
 14  resp_ip_bytes  float64
 15  label          object 
dtypes: float64(11), object(5)
memory usage: 363.6+ MB


Removing some attack lables as they are very rare .. hence it is better to remove these attacks 

In [24]:
X.drop(X[X['label'] == 'C&C-HeartBeat' ].index, inplace = True)

In [25]:
X.drop(X[X['label'] == 'C&C-Torii' ].index, inplace = True)

In [26]:
X.drop(X[X['label'] == 'Okiru-Attack' ].index, inplace = True)

Separating dependent and independent variables

In [27]:
X=X.iloc[:,:-1]

In [28]:
X.head()

,ts,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,1.536227e+09,17576.0,8081.0,tcp,-,3000000.0,0.0,0.0,S0,0.0,S,2.0,80.0,0.0,0.0
1,1.536227e+09,17576.0,8081.0,tcp,-,2000000.0,0.0,0.0,S0,0.0,S,2.0,80.0,0.0,0.0
2,1.536227e+09,17576.0,8081.0,tcp,-,2000000.0,0.0,0.0,S0,0.0,S,2.0,80.0,0.0,0.0
3,1.536227e+09,17576.0,8081.0,tcp,-,2000000.0,0.0,0.0,S0,0.0,S,2.0,80.0,0.0,0.0
4,1.536227e+09,17576.0,8081.0,tcp,-,2000000.0,0.0,0.0,S0,0.0,S,2.0,80.0,0.0,0.0


In [29]:
y=df['label']

In [30]:
y.head()

0    POHScan
1    POHScan
2    POHScan
3    POHScan
4    POHScan
Name: label, dtype: object

In [31]:
y.value_counts()

POHScan    1324265
Okiru       549228
Benign      472319
DDoS        448703
C&C           8240
Name: label, dtype: int64

let us check the categorical variables

In [66]:
cat_df =X.select_dtypes(include=['object']).copy()

In [67]:
cat_df.head()

,proto,conn_state,history
0,tcp,S0,S
1,tcp,S0,S
2,tcp,S0,S
3,tcp,S0,S
4,tcp,S0,S


In [69]:
num_df =X.select_dtypes(exclude=['object']).copy()

In [70]:
num_df.head()

,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,1.536227e+09,17576.0,8081.0,3000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0
1,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0
2,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0
3,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0
4,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0


ok so there are only three categorical variables...

In [68]:
le=LabelEncoder()

In [72]:
enc_df =cat_df.apply(LabelEncoder().fit_transform)

In [73]:
enc_df.head()

,proto,conn_state,history
0,1,6,31
1,1,6,31
2,1,6,31
3,1,6,31
4,1,6,31


In [76]:
Xe=pd.concat([num_df,enc_df], axis=1)

In [92]:
Xe.head()

,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto,conn_state,history
0,1.536227e+09,17576.0,8081.0,3000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
1,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
2,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
3,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
4,1.536227e+09,17576.0,8081.0,2000000.0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31


In [93]:
Xe= Xe.drop(columns=['ts'])

In [108]:
Xe= Xe.drop(columns=['id.orig_p','id.resp_p','duration'])

In [109]:
Xe.head()

,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto,conn_state,history
0,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
1,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
2,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
3,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31
4,0.0,0.0,0.0,2.0,80.0,0.0,0.0,1,6,31


In [110]:
Ye=le.fit_transform(y)

In [111]:
Ye

array([4, 4, 4, ..., 0, 0, 0])

In [112]:
scaling=StandardScaler()

In [113]:
Xs=scaling.fit_transform(Xe)

In [114]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc,roc_auc_score,roc_curve,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [115]:
X_train, X_test, Y_train, Y_test = train_test_split(Xs, Ye, random_state=10, test_size=0.20)

In [116]:
X_train.shape

(1961928, 10)

In [117]:
X_test.shape

(840827, 10)

In [118]:
Y_train.shape

(1961928,)

In [119]:
Y_test.shape

(840827,)

In [120]:
train_scores, test_scores = list(), list()
values = [i for i in range(1, 15)]

In [121]:
for i in values:
	# configure the model
	model = DecisionTreeClassifier(max_depth=i)
	# fit model on the training dataset
	model.fit(X_train, Y_train)
	# evaluate on the train dataset
	train_yhat = model.predict(X_train)
	train_acc = accuracy_score(Y_train, train_yhat)
	train_scores.append(train_acc)
	# evaluate on the test dataset
	test_yhat = model.predict(X_test)
	test_acc = accuracy_score(Y_test, test_yhat)
	test_scores.append(test_acc)
	# summarize progress
	print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))

>1, train: 0.626, test: 0.625
>2, train: 0.768, test: 0.768
>3, train: 0.768, test: 0.768
>4, train: 0.768, test: 0.768
>5, train: 0.768, test: 0.768
>6, train: 0.768, test: 0.768
>7, train: 0.768, test: 0.768
>8, train: 0.768, test: 0.768
>9, train: 0.768, test: 0.768
>10, train: 0.768, test: 0.768
>11, train: 0.768, test: 0.768
>12, train: 0.768, test: 0.768
>13, train: 0.768, test: 0.768
>14, train: 0.768, test: 0.768


The model is either underfitting or overfitting there is something wrong with Data! 

In [122]:
start = timer.time()
gnb_model = GaussianNB()
gnb_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 0.69 seconds


In [123]:
# Predicting the test set results
y_gnb = gnb_model.predict(X_test)
y_gnb_prob = gnb_model.predict_proba(X_test)

In [124]:
print("Classification report for GNB: \n{}".format(classification_report(Y_test,y_gnb)))
print("Confusion matrix for GNB: \n{}".format(confusion_matrix(Y_test,y_gnb)))
print("Accuracy score for GNB: {:.2f}".format(accuracy_score(Y_test,y_gnb)))
# calculate precision, recall, and f1 scores
prec_gnb = precision_score(Y_test,y_gnb,average='weighted')
rec_gnb = recall_score(Y_test,y_gnb,average='weighted')
f1_gnb = f1_score(Y_test,y_gnb,average='weighted')
print("Precision score for GNB: {:.2f}".format(prec_gnb))
print("Recall score for GNB: {:.2f}".format(rec_gnb))
print("F1 score for GNB: {:.2f}".format(f1_gnb))

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for GNB: 
              precision    recall  f1-score   support

           0       0.99      0.91      0.95    142280
           1       0.00      0.00      0.00      2406
           2       1.00      0.89      0.94    134971
           3       0.28      1.00      0.44    164846
           4       0.87      0.01      0.01    396324

    accuracy                           0.50    840827
   macro avg       0.63      0.56      0.47    840827
weighted avg       0.79      0.50      0.40    840827

Confusion matrix for GNB: 
[[129276      0     38  12623    343]
 [     5      0      0   2401      0]
 [     4      0 119927  15035      5]
 [     0      0      0 164821     25]
 [  1343      0      4 392499   2478]]
Accuracy score for GNB: 0.50


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision score for GNB: 0.79
Recall score for GNB: 0.50
F1 score for GNB: 0.40


In [130]:
from collections import Counter

In [131]:
print('Original dataset shape %s' % Counter(Ye))

Original dataset shape Counter({4: 1324265, 3: 549228, 0: 472319, 2: 448703, 1: 8240})


In [125]:
from imblearn import under_sampling, over_sampling

In [128]:
from imblearn.under_sampling import RandomUnderSampler

In [150]:
rus=RandomUnderSampler(sampling_strategy='majority',random_state=21)

In [151]:
X_res, y_res = rus.fit_resample(Xe, Ye)

In [152]:
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({3: 549228, 0: 472319, 2: 448703, 1: 8240, 4: 8240})


Followed by oversampling

In [153]:
from imblearn.over_sampling import RandomOverSampler

In [154]:
over = RandomOverSampler(sampling_strategy='minority')

In [157]:
X_rs, y_rs = over.fit_resample(X_res, y_res)

In [158]:
print('Resampled dataset shape %s' % Counter(y_rs))

Resampled dataset shape Counter({1: 549228, 3: 549228, 0: 472319, 2: 448703, 4: 8240})


In [159]:
X_train, X_test, Y_train, Y_test = train_test_split(X_res, y_res, random_state=10, test_size=0.20)

In [160]:
train_scores, test_scores = list(), list()
values = [i for i in range(1, 15)]

In [161]:
for i in values:
	# configure the model
	model = DecisionTreeClassifier(max_depth=i)
	# fit model on the training dataset
	model.fit(X_train, Y_train)
	# evaluate on the train dataset
	train_yhat = model.predict(X_train)
	train_acc = accuracy_score(Y_train, train_yhat)
	train_scores.append(train_acc)
	# evaluate on the test dataset
	test_yhat = model.predict(X_test)
	test_acc = accuracy_score(Y_test, test_yhat)
	test_scores.append(test_acc)
	# summarize progress
	print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))

>1, train: 0.638, test: 0.637
>2, train: 0.926, test: 0.926
>3, train: 0.926, test: 0.926
>4, train: 0.945, test: 0.945
>5, train: 0.955, test: 0.955
>6, train: 0.955, test: 0.955
>7, train: 0.955, test: 0.955
>8, train: 0.955, test: 0.955
>9, train: 0.955, test: 0.955
>10, train: 0.955, test: 0.955
>11, train: 0.955, test: 0.955
>12, train: 0.955, test: 0.955
>13, train: 0.955, test: 0.955
>14, train: 0.955, test: 0.955
